# Assignment #5 - Artificial Neural Networks
---
## Name: 李勝維
## Student ID: 0711239
---

In [7]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 500

# Data Input & Data Visualization

Use package 'json' to read and 'pandas' to save data  

In [8]:
import json
# Data input
with open('2021-nycu-ml-hw5\\train.json', 'r') as file:
    raw_data = json.load(file)
raw_data = [(x['id'], x['cuisine'], x['ingredients']) for x in raw_data]
ids, cuisines, ingredientss = zip(*raw_data)
df = pd.DataFrame({
    'id':ids,
    'cuisine':cuisines,
    'ingredients':ingredientss,
})

# Data Preprocessing

In [9]:
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, random_split

class myDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        super().__init__()
        self.ids = input_ids
        self.mask = attention_masks
        self.y = labels
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return {
            'attention_mask':self.mask[idx],
            'input_ids':self.ids[idx],
            'label':self.y[idx],
        }

# Shuffle data
df = df.sample(frac=1, random_state=0)

# Encode Y
y_encoder = LabelEncoder()
df['cuisine'] = y_encoder.fit_transform(df['cuisine'])
df['cuisine'].values.astype(np.longlong)

# Concat X
df['concat'] = [' '.join(s).lower() for s in df['ingredients']]

ingredients = df['concat'].tolist()
labels = df['cuisine'].values.astype(np.longlong)

# Tokenlize X
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

encoded_dict = tokenizer(ingredients, padding=True, truncation=True, return_tensors='pt')

input_ids = encoded_dict['input_ids']
attention_masks = encoded_dict['attention_mask']
labels = torch.tensor(labels)

# Transform data into torch dataloader
dataset = myDataset(input_ids, attention_masks, labels)
train_size = round(0.7 * len(dataset))

batch_size = 32
train_dataset, valid_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
# train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
# valid_loader = DataLoader(valid_dataset, shuffle=True, batch_size=batch_size)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\pha12/.cache\huggingface\transformers\45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at C:\Users\pha12/.cache\huggingface\transformers\c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at C:\Users\pha12/.cache\huggingface\transformers\534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b60

In [10]:
from transformers import BertForSequenceClassification, TrainingArguments, Trainer

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 20,
    output_attentions = False,
    output_hidden_states = False,).cuda()

training_args = TrainingArguments("weights")
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=valid_dataset
)
trainer.train()

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\pha12/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19"
  },
  "initializer_range":

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 2.00 GiB total capacity; 1.13 GiB already allocated; 0 bytes free; 1.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF